In [1]:
#importación de librerias
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
from pandas import DateOffset
import boto3
import sagemaker
from botocore.exceptions import ClientError
import logging
import matplotlib.pyplot as plt
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\pydantic\_internal\_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/25/25 13:09:07] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=517352;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=932871;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\botocore\credentials.py#1352\1352]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Usuario\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
df=pd.read_csv('data_csv/15_materiales_mod.csv',sep=',',index_col=0,parse_dates=True,decimal='.')

In [3]:
materiales = df['material'].unique()
print(f'Los materiales son {materiales}')

Los materiales son [20000337001 20000400003 20000815002 20000815003 20000837001 20000837002
 20001016001 20001374001 20003147001 20003257001 20003257002 20003257004
 20008046001 25109225001 25109232001]


In [4]:
timeseries = []
materiales = df['material'].unique()
for mat in materiales:
    serie = df[df['material'] == mat].sort_index()
    timeseries.append(serie)

In [5]:
def sumar_cantidades_fechas(lista_dataframes, fechas=['2023-11-01', '2023-11-02']):
    """
    Suma la columna 'cantidad' para las fechas especificadas en cada dataframe
    
    Args:
        lista_dataframes: Lista de dataframes con índice de fecha
        fechas: Lista de fechas a considerar (por defecto '2023-11-01' y '2023-11-02')
    
    Returns:
        Un diccionario con las sumas por cada dataframe
    """
    resultados = {}
    
    for i, df in enumerate(lista_dataframes):
        suma = 0
        for fecha in fechas:
            if fecha in df.index:
                suma += df.loc[fecha, 'cantidad']
        resultados[f'dataframe_{i}'] = suma
    
    return resultados

In [6]:
resultados = sumar_cantidades_fechas(timeseries)

In [7]:
resultados

{'dataframe_0': 33.0,
 'dataframe_1': 20.0,
 'dataframe_2': 25.0,
 'dataframe_3': 31.0,
 'dataframe_4': 28.0,
 'dataframe_5': 44.0,
 'dataframe_6': 4.0,
 'dataframe_7': 5.0,
 'dataframe_8': 133.0,
 'dataframe_9': 46.0,
 'dataframe_10': 60.0,
 'dataframe_11': 26.0,
 'dataframe_12': 43.0,
 'dataframe_13': 82.0,
 'dataframe_14': 43.0}

In [8]:
def eliminar_fechas(lista_dataframes, fechas=['2023-11-01', '2023-11-02']):
    """
    Elimina los registros de las fechas especificadas en cada dataframe
    
    Args:
        lista_dataframes: Lista de dataframes con índice de fecha
        fechas: Lista de fechas a eliminar (por defecto '2023-11-01' y '2023-11-02')
    
    Returns:
        Una nueva lista de dataframes sin los registros de las fechas indicadas
    """
    nuevos_dataframes = []
    
    for df in lista_dataframes:
        # Crear una copia para no modificar el original
        df_nuevo = df.copy()
        # Eliminar las fechas especificadas si existen en el índice
        df_nuevo = df_nuevo.drop(fechas, errors='ignore')
        nuevos_dataframes.append(df_nuevo)
    
    return nuevos_dataframes

In [9]:
timeseries = eliminar_fechas(timeseries)

In [10]:
def agrupar_por_mes(lista_dataframes):
    """
    Agrupa los datos por mes-año, sumando la columna 'cantidad' y manteniendo las columnas categóricas.
    
    Args:
        lista_dataframes: Lista de dataframes con índice de fecha en formato 'YYYY-MM-DD'
    
    Returns:
        Lista de dataframes con datos agrupados por mes-año
    """
    import pandas as pd
    
    dataframes_agrupados = []
    
    for df in lista_dataframes:
        # Verificar que el índice sea de tipo datetime
        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)
        
        # Crear una columna con el mes-año para agrupar
        df_copia = df.copy()
        df_copia['mes_anio'] = df_copia.index.strftime('%Y-%m')
        
        # Identificar columnas categóricas (todas excepto 'cantidad')
        cols_categoricas = [col for col in df_copia.columns if col != 'cantidad' and col != 'mes_anio']
        
        # Realizar la agrupación
        if cols_categoricas:
            # Si hay columnas categóricas, agrupar por mes_anio y columnas categóricas
            grupos = df_copia.groupby(['mes_anio'] + cols_categoricas)
            # Sumar la cantidad para cada grupo
            agrupado = grupos['cantidad'].sum().reset_index()
            # Convertir 'mes_anio' en índice de datetime (primer día del mes)
            agrupado['fecha'] = pd.to_datetime(agrupado['mes_anio'] + '-01')
            agrupado = agrupado.set_index('fecha')
            agrupado = agrupado.drop('mes_anio', axis=1)
        else:
            # Si no hay columnas categóricas, agrupar solo por mes_anio
            agrupado = df_copia.groupby('mes_anio')['cantidad'].sum().reset_index()
            agrupado['fecha'] = pd.to_datetime(agrupado['mes_anio'] + '-01')
            agrupado = agrupado.set_index('fecha')
            agrupado = agrupado.drop('mes_anio', axis=1)
        
        dataframes_agrupados.append(agrupado)
    
    return dataframes_agrupados

In [11]:
timeseries = agrupar_por_mes(timeseries)

In [12]:
def eliminar_ultimo_registro(dataframes_agrupados):
    """
    Elimina el último registro de cada dataframe en la lista.
    
    Args:
        dataframes_agrupados: Lista de dataframes con datos agrupados por mes-año
    
    Returns:
        Lista de dataframes con el último registro eliminado en cada uno
    """
    resultado = []
    
    for df in dataframes_agrupados:
        # Si el dataframe tiene al menos un registro
        if len(df) > 0:
            # Eliminar el último registro
            df_sin_ultimo = df.iloc[:-1].copy()
            resultado.append(df_sin_ultimo)
        else:
            # Si el dataframe está vacío, agregarlo sin cambios
            resultado.append(df.copy())
    
    return resultado

In [13]:
timeseries = eliminar_ultimo_registro(timeseries)

In [14]:
def ajustar_para_mantener_ratios(timeseries, resultados):
    """
    Ajusta los valores de 2023-09 y 2023-10 sumando cantidades para que mantengan 
    los mismos ratios que 2022-09 y 2022-10, utilizando los valores de 'resultados'.
    
    Args:
        timeseries: Lista de dataframes mensuales
        resultados: Diccionario con las sumas de cantidades
    
    Returns:
        Lista de dataframes con las cantidades ajustadas
    """
    import pandas as pd
    import numpy as np
    
    dataframes_ajustados = []
    
    for i, df in enumerate(timeseries):
        df_ajustado = df.copy()
        clave_resultado = f'dataframe_{i}'
        
        # Verificar que exista el valor en resultados
        if clave_resultado in resultados:
            valor_a_distribuir = resultados[clave_resultado]
            
            # 1. Calcular los ratios objetivo de 2022-09 y 2022-10
            if '2022-09-01' in df_ajustado.index and '2022-10-01' in df_ajustado.index:
                valor_sep_2022 = df_ajustado.loc['2022-09-01', 'cantidad']
                valor_oct_2022 = df_ajustado.loc['2022-10-01', 'cantidad']
                
                # Ratio objetivo (proporción de sep respecto a la suma)
                if valor_sep_2022 + valor_oct_2022 > 0:
                    ratio_objetivo = valor_sep_2022 / (valor_sep_2022 + valor_oct_2022)
                else:
                    ratio_objetivo = 0.5
                
                # Valores actuales o cero si no existen
                valor_sep_2023 = df_ajustado.loc['2023-09-01', 'cantidad'] if '2023-09-01' in df_ajustado.index else 0
                valor_oct_2023 = df_ajustado.loc['2023-10-01', 'cantidad'] if '2023-10-01' in df_ajustado.index else 0
                
                # 2. Calcular cuánto agregar a cada mes para mantener el ratio objetivo
                # Resolviendo el sistema de ecuaciones:
                # (valor_sep_2023 + x) / (valor_sep_2023 + x + valor_oct_2023 + y) = ratio_objetivo
                # x + y = valor_a_distribuir
                
                # Si ambos valores son cero, distribuimos según el ratio objetivo
                if valor_sep_2023 == 0 and valor_oct_2023 == 0:
                    incremento_sep = valor_a_distribuir * ratio_objetivo
                    incremento_oct = valor_a_distribuir * (1 - ratio_objetivo)
                else:
                    # Para mantener el ratio: (a + x) / (a + b + x + y) = r, donde x + y = v
                    # Resolviendo: x = (r*(a+b) - a) + r*v
                    a = valor_sep_2023
                    b = valor_oct_2023
                    r = ratio_objetivo
                    v = valor_a_distribuir
                    
                    # Calculamos los incrementos
                    incremento_sep = max(0, (r*(a+b) - a) + r*v)
                    incremento_oct = max(0, v - incremento_sep)
                    
                    # Si hay valores negativos, ajustamos la distribución
                    if incremento_sep < 0 or incremento_oct < 0:
                        incremento_sep = v * r
                        incremento_oct = v * (1 - r)
                
                # Actualizar o crear los registros para 2023-09
                if '2023-09-01' in df_ajustado.index:
                    # Sumar el incremento calculado
                    df_ajustado.loc['2023-09-01', 'cantidad'] += incremento_sep
                else:
                    # Crear nuevo registro
                    nuevo_registro = pd.DataFrame(index=[pd.to_datetime('2023-09-01')])
                    
                    # Copiar valores categóricos
                    if len(df_ajustado) > 0:
                        for col in df_ajustado.columns:
                            if col != 'cantidad':
                                nuevo_registro[col] = df_ajustado[col].iloc[0]
                    
                    # Asignar el valor calculado
                    nuevo_registro['cantidad'] = incremento_sep
                    
                    # Concatenar
                    df_ajustado = pd.concat([df_ajustado, nuevo_registro])
                
                # Actualizar o crear los registros para 2023-10
                if '2023-10-01' in df_ajustado.index:
                    # Sumar el incremento calculado
                    df_ajustado.loc['2023-10-01', 'cantidad'] += incremento_oct
                else:
                    # Crear nuevo registro
                    nuevo_registro = pd.DataFrame(index=[pd.to_datetime('2023-10-01')])
                    
                    # Copiar valores categóricos
                    if len(df_ajustado) > 0:
                        for col in df_ajustado.columns:
                            if col != 'cantidad':
                                nuevo_registro[col] = df_ajustado[col].iloc[0]
                    
                    # Asignar el valor calculado
                    nuevo_registro['cantidad'] = incremento_oct
                    
                    # Concatenar
                    df_ajustado = pd.concat([df_ajustado, nuevo_registro])
                
                # Ordenar por fecha
                df_ajustado = df_ajustado.sort_index()
        
        dataframes_ajustados.append(df_ajustado)
    
    return dataframes_ajustados

In [15]:
timeseries = ajustar_para_mantener_ratios(timeseries,resultados)

In [16]:
def agregar_columnas_temporales(lista_dataframes):
    """
    Agrega las columnas 'month' y 'quarter' a cada dataframe, calculadas a partir del índice de fecha.
    
    Args:
        lista_dataframes: Lista de dataframes con índice de fecha
    
    Returns:
        Lista de dataframes con las nuevas columnas 'month' y 'quarter'
    """
    import pandas as pd
    
    resultado = []
    
    for df in lista_dataframes:
        # Crear una copia del dataframe
        df_nuevo = df.copy()
        
        # Asegurarse de que el índice es de tipo datetime
        if not isinstance(df_nuevo.index, pd.DatetimeIndex):
            df_nuevo.index = pd.to_datetime(df_nuevo.index)
        
        # Agregar columna de mes (valores 1-12)
        df_nuevo['month'] = df_nuevo.index.month
        
        # Agregar columna de trimestre (valores 1-4)
        df_nuevo['quarter'] = df_nuevo.index.quarter
        
        # Añadir a la lista de resultados
        resultado.append(df_nuevo)
    
    return resultado

In [17]:
timeseries = agregar_columnas_temporales(timeseries)

In [18]:
def codificar_columnas_categoricas(lista_dataframes):
    """
    Codifica las columnas categóricas usando códigos numéricos únicos para cada valor.
    Las columnas a codificar son: Tipo_Producto, segmento_producto, supergrupo_producto, 
    grupo_producto y subgrupo_producto.
    
    Args:
        lista_dataframes: Lista de dataframes con columnas categóricas
    
    Returns:
        Lista de dataframes con columnas categóricas codificadas y diccionario de mapeos
    """
    import pandas as pd
    
    # Columnas categóricas a codificar
    columnas_categoricas = [
        'Tipo_Producto', 
        'segmento_producto', 
        'supergrupo_producto', 
        'grupo_producto', 
        'subgrupo_producto'
    ]
    
    # Diccionario para almacenar los mapeos para cada columna
    mapeos = {col: {} for col in columnas_categoricas}
    
    # Lista para almacenar los dataframes codificados
    dataframes_codificados = []
    
    # Crear mapeos para cada columna categórica
    codigo_actual = {}
    for col in columnas_categoricas:
        codigo_actual[col] = 0
    
    # Iterar sobre cada dataframe para crear los mapeos
    for df in lista_dataframes:
        for col in columnas_categoricas:
            if col in df.columns:
                # Obtener el valor único en esta columna para este dataframe
                # (asumiendo que cada dataframe tiene un solo valor para cada columna categórica)
                if len(df) > 0:
                    valor = df[col].iloc[0]
                    
                    # Si el valor no está en el mapeo, asignarle un código
                    if valor not in mapeos[col]:
                        mapeos[col][valor] = codigo_actual[col]
                        codigo_actual[col] += 1
    
    # Aplicar los mapeos a cada dataframe
    for df in lista_dataframes:
        df_codificado = df.copy()
        
        for col in columnas_categoricas:
            if col in df.columns:
                if len(df) > 0:
                    valor = df[col].iloc[0]
                    codigo = mapeos[col][valor]
                    
                    # Reemplazar el valor categórico con su código
                    df_codificado[col] = codigo
        
        dataframes_codificados.append(df_codificado)
    
    return dataframes_codificados, mapeos

In [19]:
timeseries,mapeos = codificar_columnas_categoricas(timeseries)

In [20]:
mapeos

{'Tipo_Producto': {'LILI PINK': 0, 'YOI': 1},
 'segmento_producto': {'JUVENIL': 0, 'TEEN': 1, 'ADULTO': 2, 'GIRLS': 3},
 'supergrupo_producto': {'INTERIOR JUVENIL': 0,
  'INTERIOR TEEN': 1,
  'LINEA SECRET': 2,
  'INTERIOR MUJER': 3,
  'BELLEZA Y BIENESTAR': 4},
 'grupo_producto': {'PANTY PAQX3': 0,
  'BRASIER PAQX2': 1,
  'PANTY PAQX2': 2,
  'PANTY': 3,
  'BRASIER SILICONA': 4,
  'FRAGANCIAS': 5},
 'subgrupo_producto': {'ALGODON': 0,
  'MICROFIBRA': 1,
  'SEAMLESS': 2,
  'ENCAJE': 3,
  'SILICONA': 4,
  'NO APLICA': 5}}

In [21]:
def verificar_registros_mensuales(lista_dataframes):
    """
    Verifica que existan registros para todos los meses desde la primera hasta la última fecha
    en cada dataframe, e identifica los meses faltantes.
    
    Args:
        lista_dataframes: Lista de dataframes con índice de fecha en formato 'YYYY-MM-DD'
    
    Returns:
        Lista de tuplas (dataframe_id, completo, meses_faltantes) donde:
        - dataframe_id: Índice del dataframe en la lista
        - completo: Boolean indicando si tiene todos los meses
        - meses_faltantes: Lista de fechas de meses faltantes en formato 'YYYY-MM-01'
    """
    import pandas as pd
    from datetime import datetime
    
    resultados = []
    
    for i, df in enumerate(lista_dataframes):
        # Asegurarse de que el índice es de tipo datetime
        if not isinstance(df.index, pd.DatetimeIndex):
            df = df.copy()
            df.index = pd.to_datetime(df.index)
        
        # Verificar que hay registros en el dataframe
        if len(df) == 0:
            resultados.append((i, True, []))
            continue
        
        # Obtener la primera y última fecha
        primera_fecha = df.index.min()
        ultima_fecha = df.index.max()
        
        # Crear un rango de todos los meses entre la primera y última fecha
        todos_los_meses = pd.date_range(
            start=pd.Timestamp(year=primera_fecha.year, month=primera_fecha.month, day=1),
            end=pd.Timestamp(year=ultima_fecha.year, month=ultima_fecha.month, day=1),
            freq='MS'  # Inicio de mes (Month Start)
        )
        
        # Convertir los índices del dataframe a inicio de mes
        meses_presentes = df.index.to_period('M').to_timestamp()
        meses_presentes = meses_presentes.unique()  # Eliminar duplicados
        
        # Encontrar meses faltantes
        meses_faltantes = [fecha for fecha in todos_los_meses if fecha not in meses_presentes]
        
        # Determinar si está completo
        completo = len(meses_faltantes) == 0
        
        # Guardar resultados
        resultados.append((i, completo, meses_faltantes))
    
    return resultados

In [22]:
verificar_registros_mensuales(timeseries)

[(0, True, []),
 (1, True, []),
 (2, True, []),
 (3, True, []),
 (4, True, []),
 (5, True, []),
 (6, True, []),
 (7, True, []),
 (8, True, []),
 (9, True, []),
 (10, True, []),
 (11, True, []),
 (12, True, []),
 (13, True, []),
 (14, True, [])]

In [23]:
def extraer_vectores_categoricos(lista_dataframes):
    """
    Extrae los valores categóricos del primer registro de cada dataframe y crea
    un vector con estos valores.
    
    Args:
        lista_dataframes: Lista de dataframes que contienen columnas categóricas
    
    Returns:
        Lista de vectores categóricos, uno por cada dataframe
    """
    # Definir las columnas categóricas
    columnas_categoricas = [
        'Tipo_Producto', 
        'segmento_producto', 
        'supergrupo_producto', 
        'grupo_producto', 
        'subgrupo_producto'
    ]
    
    vectores_categoricos = []
    
    for i, df in enumerate(lista_dataframes):
        # Verificar que el dataframe tenga registros
        if len(df) == 0:
            print(f"Advertencia: Dataframe {i} está vacío, se usará un vector de ceros.")
            vectores_categoricos.append([0] * len(columnas_categoricas))
            continue
        
        # Crear el vector para este dataframe
        vector = []
        
        for col in columnas_categoricas:
            # Verificar si la columna existe en el dataframe
            if col in df.columns:
                # Obtener el valor del primer registro
                valor = df[col].iloc[0]
                
                # Convertir a entero si es posible, de lo contrario usar un código hash
                try:
                    valor_numerico = int(valor)
                except (ValueError, TypeError):
                    # Si no se puede convertir a entero, usar un código hash simple
                    if valor is None:
                        valor_numerico = 0
                    else:
                        # Hash simple basado en la representación string del valor
                        valor_numerico = hash(str(valor)) % 10000  # Limitar a 4 dígitos
            else:
                # Si la columna no existe, usar 0
                valor_numerico = 0
            
            vector.append(valor_numerico)
        
        vectores_categoricos.append(vector)
        
    return vectores_categoricos

In [24]:
vectores_cat = extraer_vectores_categoricos(timeseries)

In [25]:
def extraer_vectores_cantidad(lista_dataframes):
    """
    Extrae los valores de la columna 'cantidad' de cada dataframe y crea 
    un vector con estos valores.
    
    Args:
        lista_dataframes: Lista de dataframes que contienen la columna 'cantidad'
    
    Returns:
        Lista de vectores, donde cada vector contiene los valores de 'cantidad' de un dataframe
    """
    import pandas as pd
    import numpy as np
    
    vectores_cantidad = []
    
    for i, df in enumerate(lista_dataframes):
        # Verificar que el dataframe tenga registros
        if len(df) == 0:
            print(f"Advertencia: Dataframe {i} está vacío.")
            vectores_cantidad.append([])
            continue
        
        # Verificar que exista la columna 'cantidad'
        if 'cantidad' not in df.columns:
            print(f"Advertencia: Dataframe {i} no tiene columna 'cantidad'.")
            vectores_cantidad.append([])
            continue
        
        # Extraer los valores de 'cantidad' como una lista
        valores = df['cantidad'].tolist()
        
        # Opcionalmente, puedes manejar valores NaN
        # valores = [0 if pd.isna(x) else x for x in valores]  # Convierte NaN a 0
        # O simplemente:
        valores = df['cantidad'].fillna(0).tolist()  # Rellena NaN con 0
        
        vectores_cantidad.append(valores)
    
    return vectores_cantidad

In [26]:
vectores_target = extraer_vectores_cantidad(timeseries)

In [27]:
def extraer_vectores_temporales(lista_dataframes):
    """
    Extrae los valores de las columnas 'month' y 'quarter' de cada dataframe
    y crea un conjunto de vectores con estos valores.
    
    Args:
        lista_dataframes: Lista de dataframes que contienen las columnas 'month' y 'quarter'
    
    Returns:
        Lista de conjuntos de vectores, donde cada conjunto contiene los vectores
        de 'month' y 'quarter' para un dataframe
    """
    import pandas as pd
    import numpy as np
    
    conjuntos_vectores = []
    
    for i, df in enumerate(lista_dataframes):
        # Verificar que el dataframe tenga registros
        if len(df) == 0:
            print(f"Advertencia: Dataframe {i} está vacío.")
            conjuntos_vectores.append(([], []))
            continue
        
        # Verificar que existan las columnas necesarias
        columnas_faltantes = []
        if 'month' not in df.columns:
            columnas_faltantes.append('month')
        if 'quarter' not in df.columns:
            columnas_faltantes.append('quarter')
        
        if columnas_faltantes:
            print(f"Advertencia: Dataframe {i} no tiene las columnas: {columnas_faltantes}")
            
            # Si faltan las columnas, podemos crearlas a partir del índice si es de tipo fecha
            df_temp = df.copy()
            
            if isinstance(df_temp.index, pd.DatetimeIndex):
                if 'month' not in df_temp.columns:
                    df_temp['month'] = df_temp.index.month
                if 'quarter' not in df_temp.columns:
                    df_temp['quarter'] = df_temp.index.quarter
            else:
                # Si el índice no es de tipo fecha, tratar de convertirlo
                try:
                    df_temp.index = pd.to_datetime(df_temp.index)
                    if 'month' not in df_temp.columns:
                        df_temp['month'] = df_temp.index.month
                    if 'quarter' not in df_temp.columns:
                        df_temp['quarter'] = df_temp.index.quarter
                except:
                    # Si no se puede convertir, usar valores vacíos
                    month_vector = []
                    quarter_vector = []
                    conjuntos_vectores.append((month_vector, quarter_vector))
                    continue
            
            # Usar el dataframe temporal con las columnas agregadas
            df = df_temp
        
        # Extraer los vectores
        month_vector = df['month'].tolist()
        quarter_vector = df['quarter'].tolist()
        
        # Guardar el conjunto de vectores
        conjuntos_vectores.append((month_vector, quarter_vector))
    
    return conjuntos_vectores

In [28]:
vectores_dynamic = extraer_vectores_temporales(timeseries)

In [29]:
def extraer_primeros_indices(lista_dataframes):
    """
    Extrae el primer índice de cada dataframe y lo devuelve en formato "YYYY-MM-DD 00:00:00".
    
    Args:
        lista_dataframes: Lista de dataframes con índices de fecha
    
    Returns:
        Lista de strings con los primeros índices en formato "YYYY-MM-DD 00:00:00"
    """
    import pandas as pd
    from datetime import datetime
    
    primeros_indices = []
    
    for i, df in enumerate(lista_dataframes):
        # Verificar que el dataframe tenga registros
        if len(df) == 0:
            print(f"Advertencia: Dataframe {i} está vacío. Se usará fecha por defecto.")
            primeros_indices.append("2000-01-01 00:00:00")
            continue
        
        # Obtener el primer índice
        primer_indice = df.index[0]
        
        # Convertir a datetime si no lo es
        if not isinstance(primer_indice, pd.Timestamp) and not isinstance(primer_indice, datetime):
            try:
                primer_indice = pd.to_datetime(primer_indice)
            except:
                print(f"Advertencia: No se pudo convertir el índice del Dataframe {i} a fecha. Se usará fecha por defecto.")
                primeros_indices.append("2000-01-01 00:00:00")
                continue
        
        # Formatear a "YYYY-MM-DD 00:00:00"
        indice_formateado = primer_indice.strftime("%Y-%m-%d 00:00:00")
        
        primeros_indices.append(indice_formateado)
    
    return primeros_indices

In [30]:
start = extraer_primeros_indices(timeseries)

In [31]:
def crear_diccionarios_test(start, vectores_target, vectores_cat, vectores_dynamic):
    """
    Crea una lista de diccionarios con la estructura requerida para entrenamiento,
    donde start son las fechas de inicio de cada serie (un valor por dataframe).
    
    Args:
        start: Lista de strings con las fechas de inicio (un valor por dataframe)
        vectores_target: Lista de vectores con los valores de 'cantidad' para cada serie
        vectores_cat: Lista de vectores con las características categóricas
        vectores_dynamic: Lista de tuplas (month_vector, quarter_vector) con características dinámicas
    
    Returns:
        Lista de diccionarios con la estructura {start, target, cat, dynamic_feat}
    """
    import numpy as np
    
    def convert_nans_to_string(values_list):
        """Convierte valores NaN en la lista a 'NaN' como string"""
        return ['NaN' if (isinstance(x, float) and np.isnan(x)) else float(x) for x in values_list]
    
    diccionarios = []
    
    # Verificar que tengamos el mismo número de series en todas las listas
    num_series = len(start)
    if not (num_series == len(vectores_target) == len(vectores_cat) == len(vectores_dynamic)):
        print(f"Error: Las listas tienen diferentes longitudes - start: {len(start)}, target: {len(vectores_target)}, " 
              f"cat: {len(vectores_cat)}, dynamic: {len(vectores_dynamic)}")
        return []
    
    for i in range(num_series):
        # Verificar que haya datos para esta serie
        if not vectores_target[i]:
            print(f"Advertencia: Serie {i} no tiene valores target. Se omitirá.")
            continue
        
        # Obtener los datos para esta serie
        fecha_inicio = start[i]
        target_data = vectores_target[i]
        cat_data = vectores_cat[i]
        month_vector, quarter_vector = vectores_dynamic[i]
        
        # Verificar longitudes de vectors target y dynamic
        if len(target_data) != len(month_vector) or len(target_data) != len(quarter_vector):
            print(f"Advertencia: Serie {i} tiene longitudes inconsistentes - target: {len(target_data)}, "
                  f"month: {len(month_vector)}, quarter: {len(quarter_vector)}")
        
        # Crear el diccionario
        diccionario = {
            "start": fecha_inicio,
            "target": convert_nans_to_string(target_data),
            "cat": cat_data,
            "dynamic_feat": [month_vector, quarter_vector]  # Usar valores originales sin normalizar
        }
        
        diccionarios.append(diccionario)
    
    return diccionarios

In [32]:
test = crear_diccionarios_test(start,vectores_target,vectores_cat,vectores_dynamic)

In [33]:
def crear_diccionarios_entrenamiento(start, vectores_target, vectores_cat, vectores_dynamic, puntos_a_excluir=6):
    """
    Crea una lista de diccionarios excluyendo los últimos 'puntos_a_excluir' valores de 
    target y dynamic_feat para cada serie.
    
    Args:
        start: Lista de strings con las fechas de inicio (un valor por dataframe)
        vectores_target: Lista de vectores con los valores de 'cantidad' para cada serie
        vectores_cat: Lista de vectores con las características categóricas
        vectores_dynamic: Lista de tuplas (month_vector, quarter_vector) con características dinámicas
        puntos_a_excluir: Número de puntos a excluir del final de las series (default=6)
    
    Returns:
        Lista de diccionarios con la estructura {start, target, cat, dynamic_feat}
    """
    import numpy as np
    
    def convert_nans_to_string(values_list):
        """Convierte valores NaN en la lista a 'NaN' como string"""
        return ['NaN' if (isinstance(x, float) and np.isnan(x)) else float(x) for x in values_list]
    
    diccionarios = []
    
    # Verificar que tengamos el mismo número de series en todas las listas
    num_series = len(start)
    if not (num_series == len(vectores_target) == len(vectores_cat) == len(vectores_dynamic)):
        print(f"Error: Las listas tienen diferentes longitudes - start: {len(start)}, target: {len(vectores_target)}, " 
              f"cat: {len(vectores_cat)}, dynamic: {len(vectores_dynamic)}")
        return []
    
    for i in range(num_series):
        # Verificar que haya datos para esta serie
        if not vectores_target[i]:
            print(f"Advertencia: Serie {i} no tiene valores target. Se omitirá.")
            continue
        
        # Obtener los datos para esta serie
        fecha_inicio = start[i]
        target_data = vectores_target[i]
        cat_data = vectores_cat[i]
        month_vector, quarter_vector = vectores_dynamic[i]
        
        # Verificar que hay suficientes puntos para excluir
        if len(target_data) <= puntos_a_excluir:
            print(f"Advertencia: Serie {i} tiene menos puntos ({len(target_data)}) que los requeridos a excluir ({puntos_a_excluir}). Se omitirá.")
            continue
        
        # Excluir los últimos 'puntos_a_excluir' valores
        target_data_recortado = target_data[:-puntos_a_excluir]
        month_vector_recortado = month_vector[:-puntos_a_excluir]
        quarter_vector_recortado = quarter_vector[:-puntos_a_excluir]
        
        # Verificar longitudes de vectors target y dynamic después del recorte
        if len(target_data_recortado) != len(month_vector_recortado) or len(target_data_recortado) != len(quarter_vector_recortado):
            print(f"Advertencia: Serie {i} tiene longitudes inconsistentes después del recorte - target: {len(target_data_recortado)}, "
                  f"month: {len(month_vector_recortado)}, quarter: {len(quarter_vector_recortado)}")
            # Ajustar a la longitud mínima
            min_len = min(len(target_data_recortado), len(month_vector_recortado), len(quarter_vector_recortado))
            target_data_recortado = target_data_recortado[:min_len]
            month_vector_recortado = month_vector_recortado[:min_len]
            quarter_vector_recortado = quarter_vector_recortado[:min_len]
        
        # Crear el diccionario
        diccionario = {
            "start": fecha_inicio,
            "target": convert_nans_to_string(target_data_recortado),
            "cat": cat_data,
            "dynamic_feat": [month_vector_recortado, quarter_vector_recortado]
        }
        
        diccionarios.append(diccionario)
    
    return diccionarios

In [34]:
train = crear_diccionarios_entrenamiento(start,vectores_target, vectores_cat, vectores_dynamic)

In [35]:
def write_dicts_to_file(path, data):
    with open(path, "wb") as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode("utf-8"))

In [ ]:
%%time
write_dicts_to_file("data_json/mensual_modificado_float/train.json", train)
write_dicts_to_file("data_json/mensual_modificado_float/test.json", test)

CPU times: total: 0 ns
Wall time: 2.21 ms


In [37]:
boto_session = boto3.Session(profile_name='lilipink', region_name='us-east-1')
sagemaker_session = sagemaker.Session(boto_session=boto_session)
s3_client = boto_session.client('s3')
sm_client= boto_session.client('sagemaker')
s3 = boto_session.resource("s3")

[05/22/25 14:19:17] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=589464;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=262519;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\botocore\credentials.py#1352\1352]8;;\

In [38]:
def create_bucket(bucket_name, region=None):
    try:
        if region is None:
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            location = {'LocationConstraint': region}
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration=location
            )
        print(f"Bucket S3 '{bucket_name}' creado exitosamente en {region if region else 'la región por defecto'}")
        return True
    except ClientError as e:
        logging.error(e)
        print(f"Error al crear el bucket S3: {e}")
        return False

In [39]:
bucket_name = "forecasting-mensual-15-v3"
create_bucket(bucket_name)

Bucket S3 'forecasting-mensual-15-v3' creado exitosamente en la región por defecto


True

In [40]:
s3_bucket = bucket_name  # replace with an existing bucket if needed
s3_bucket_prefix = (
        "lilipink"  
    )
default_bucket_prefix = sagemaker_session.default_bucket_prefix
if default_bucket_prefix:
    s3_prefix = f"{default_bucket_prefix}/{s3_bucket_prefix}"
else:
    s3_prefix = s3_bucket_prefix

role = "arn:aws:iam::844598627082:role/service-role/AmazonSageMaker-ExecutionRole-20250513T105052"  # IAM role to use by SageMaker
region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output".format(s3_bucket, s3_prefix)

In [41]:
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", region)

[05/22/25 14:19:18] INFO     Same images used for training and inference. Defaulting to image     ]8;id=379881;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=207173;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\image_uris.py#393\393]8;;\
                             scope: inference.                                                                     

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=959484;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=867560;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\image_uris.py#530\530]8;;\

In [42]:
def copy_to_s3(local_file, s3_path, override=False):
    assert s3_path.startswith("s3://")
    split = s3_path.split("/")
    bucket = split[2]
    path = "/".join(split[3:])
    buk = s3.Bucket(bucket)

    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print(
                "File s3://{}/{} already exists.\nSet override to upload anyway.\n".format(
                    s3_bucket, s3_path
                )
            )
            return
        else:
            print("Overwriting existing file")
    with open(local_file, "rb") as data:
        print("Uploading file to {}".format(s3_path))
        buk.put_object(Key=path, Body=data)

In [ ]:
local_file = 'data_json/mensual_modificado_float/'
copy_to_s3(local_file + 'train.json', s3_data_path + "/train/train.json",override=True)
copy_to_s3(local_file + 'test.json', s3_data_path + "/test/test.json",override=True)

Overwriting existing file
Uploading file to s3://forecasting-mensual-15-v3/lilipink/data/train/train.json
Overwriting existing file
Uploading file to s3://forecasting-mensual-15-v3/lilipink/data/test/test.json


In [44]:
estimator = sagemaker.estimator.Estimator(
    image_uri=image_name,
    sagemaker_session=sagemaker_session,
    role=role,
    instance_count=1,
    instance_type="ml.c4.2xlarge",
   # use_spot_instances=True,
   # max_run=1800,  # max training time in seconds
   # max_wait=1800,  # seconds to wait for spot instance
    base_job_name="lilipink-forecasting",
    output_path=s3_output_path,
)

[05/22/25 14:19:20] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=511139;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=633370;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\botocore\credentials.py#1352\1352]8;;\

In [45]:
###PRIMER ENTRENAMIENTO###

In [46]:
freq= "M"
context_length = 12
prediction_length = 6
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    #"learning_rate": "1E-3",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
}

In [327]:
estimator.set_hyperparameters(**hyperparameters)

In [263]:
%%time
data_channels = {"train": "{}/train/".format(s3_data_path), "test": "{}/test/".format(s3_data_path)}

estimator.fit(inputs=data_channels, wait=True)

[05/21/25 10:46:54] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=706920;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=688576;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=659238;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=827897;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#1042\1042]8;;\
                             lilipink-forecasting-2025-05-21-15-46-54-277                                          

2025-05-21 15:46:56 Starting - Starting the training job...
2025-05-21 15:47:21 Starting - Preparing the instances for training...
2025-05-21 15:47:59 Downloading - Downloading the training image...............
2025-05-21 15:50:21 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
Running custom environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
[05/21/2025 15:50:37 INFO 140269512570688] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'_kvstore': 'auto', '_num_gpus': 'auto', '_num_kv_servers': 'auto', '_tuning_objective_metric': '', 'cardinality': 'auto', 'dropout_rate': '0.10', 'early_stopping_patience': '', 'embedding_dimension': '10', 'learning_rate': '0.001', 'l

In [328]:
estimator = sagemaker.estimator.Estimator.attach('lilipink-forecasting-2025-05-21-15-46-54-277',sagemaker_session=sagemaker_session)


2025-05-21 15:55:30 Starting - Preparing the instances for training
2025-05-21 15:55:30 Downloading - Downloading the training image
2025-05-21 15:55:30 Training - Training image download completed. Training in progress.
2025-05-21 15:55:30 Uploading - Uploading generated training model
2025-05-21 15:55:30 Completed - Training job completed


In [47]:
from sagemaker.serializers import IdentitySerializer

In [48]:
class DeepARPredictor(sagemaker.predictor.Predictor):
    def __init__(self, *args, **kwargs):
        super().__init__(
            *args,
            # serializer=JSONSerializer(),
            serializer=IdentitySerializer(content_type="application/json"),
            **kwargs,
        )

    def predict(
        self,
        ts,
        cat=None,
        dynamic_feat=None,
        num_samples=100,
        return_samples=False,
        quantiles=["0.1", "0.5", "0.9"],
    ):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.

        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])

        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + ts.index.freq
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)

    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(
            ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None
        )

        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles,
        }

        http_request_data = {"instances": [instance], "configuration": configuration}

        return json.dumps(http_request_data).encode("utf-8")

    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode("utf-8"))["predictions"][0]
        prediction_length = len(next(iter(predictions["quantiles"].values())))
        prediction_index = pd.date_range(
            start=prediction_time, freq=freq, periods=prediction_length
        )
        if return_samples:
            dict_of_samples = {"sample_" + str(i): s for i, s in enumerate(predictions["samples"])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(
            data={**predictions["quantiles"], **dict_of_samples}, index=prediction_index
        )

    def set_frequency(self, freq):
        self.freq = freq


def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]


def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat
    return obj

In [267]:
predictor = estimator.deploy(
    initial_instance_count=1, instance_type="ml.m5.large", predictor_cls=DeepARPredictor
)

[05/21/25 10:56:44] INFO     Creating model with name: lilipink-forecasting-2025-05-21-15-56-44-579 ]8;id=962564;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=345416;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4094\4094]8;;\

[05/21/25 10:56:45] INFO     Creating endpoint-config with name                                     ]8;id=862905;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=803381;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#6019\6019]8;;\
                             lilipink-forecasting-2025-05-21-15-56-44-579                                          

[05/21/25 10:56:46] INFO     Creating endpoint with name                                            ]8;id=551908;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=428340;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4841\4841]8;;\
                             lilipink-forecasting-2025-05-21-15-56-44-579                                          

-----------!

In [331]:
def convertir_a_series(timeseries):
    series_list = []
    
    for ts in timeseries:
        # Verificar que la columna 'cantidad' existe
        if 'cantidad' in ts.columns:
            # Extraer la columna 'cantidad' como una serie
            serie = ts['cantidad']

            # Asegurarse de que el índice esté ordenado
            serie = serie.sort_index()

            # Intentar inferir la frecuencia del índice
            try:
                freq = pd.infer_freq(serie.index)
                if freq is not None:
                    serie.index.freq = freq
            except Exception as e:
                print(f"No se pudo inferir frecuencia para una serie: {e}")

            series_list.append(serie)
        else:
            print(f"Advertencia: Un dataframe no contiene la columna 'cantidad'")
    
    return series_list

In [332]:
timeseries_list = convertir_a_series(timeseries)

In [333]:
def crear_lista_features_dinamicas(lista_dataframes):
    """
    Crea una lista de listas con los vectores de 'month' y 'quarter' para cada dataframe.
    
    Args:
        lista_dataframes: Lista de dataframes con columnas 'month' y 'quarter'
    
    Returns:
        Lista de listas donde cada elemento es [month_vector, quarter_vector] para un dataframe
    """
    import pandas as pd
    
    lista_features = []
    
    for i, df in enumerate(lista_dataframes):
        # Verificar que el dataframe tenga registros
        if len(df) == 0:
            print(f"Advertencia: Dataframe {i} está vacío. Se añadirá una lista vacía.")
            lista_features.append([[], []])
            continue
        
        # Verificar que existan las columnas necesarias
        columnas_faltantes = []
        if 'month' not in df.columns:
            columnas_faltantes.append('month')
        if 'quarter' not in df.columns:
            columnas_faltantes.append('quarter')
        
        if columnas_faltantes:
            # Si faltan columnas, intentar generarlas a partir del índice si es posible
            df_temp = df.copy()
                
            # Generar columnas de fechas si el índice es de tipo datetime
            if ('month' in columnas_faltantes or 'quarter' in columnas_faltantes) and isinstance(df_temp.index, pd.DatetimeIndex):
                if 'month' not in df_temp.columns:
                    df_temp['month'] = df_temp.index.month
                if 'quarter' not in df_temp.columns:
                    df_temp['quarter'] = df_temp.index.quarter
                df = df_temp
            elif 'month' in columnas_faltantes or 'quarter' in columnas_faltantes:
                # Intentar convertir el índice a datetime si no lo es
                try:
                    df_temp.index = pd.to_datetime(df_temp.index)
                    if 'month' not in df_temp.columns:
                        df_temp['month'] = df_temp.index.month
                    if 'quarter' not in df_temp.columns:
                        df_temp['quarter'] = df_temp.index.quarter
                    df = df_temp
                except:
                    print(f"Error: No se pueden generar las columnas {columnas_faltantes} para el Dataframe {i}. Se añadirá una lista vacía.")
                    lista_features.append([[], []])
                    continue
        
        # Crear los vectores de características dinámicas
        month_vector = df['month'].tolist()
        quarter_vector = df['quarter'].tolist()
        
        # Añadir los vectores a la lista
        lista_features.append([month_vector, quarter_vector])
        
    return lista_features

In [334]:
dynamic_list = crear_lista_features_dinamicas(timeseries)

In [272]:
horizon_pred=6
i=6
predictor.predict(
    ts = timeseries_list[i][:-horizon_pred], 
    cat=vectores_cat[i],
    dynamic_feat=dynamic_list[i],
    quantiles=[0.1, 0.5, 0.9],
)

,0.1,0.5,0.9
2024-11-01,8.005324,10.983337,13.424570
2024-12-01,0.119590,2.713603,4.847787
2025-01-01,-4.027497,-1.945598,0.712716
2025-02-01,-9.014154,-6.940654,-4.373694
2025-03-01,-5.861849,2.172155,7.716484
2025-04-01,-13.019146,1.216626,18.418119


In [273]:
timeseries_list[6].tail(6)

2024-11-01    14.0
2024-12-01    73.0
2025-01-01    26.0
2025-02-01    12.0
2025-03-01    23.0
2025-04-01    29.0
Freq: MS, Name: cantidad, dtype: float64

In [63]:
### 2DO ENTRENAMIENTO ###

In [335]:
freq= "M"
context_length = 18
prediction_length = 6
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    #"learning_rate": "1E-3",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
}

In [336]:
estimator.set_hyperparameters(**hyperparameters)

In [337]:
%%time
data_channels = {"train": "{}/train/".format(s3_data_path), "test": "{}/test/".format(s3_data_path)}

estimator.fit(inputs=data_channels, wait=True)

[05/21/25 11:14:41] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=955046;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=565503;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=616357;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=837267;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#1042\1042]8;;\
                             lilipink-forecasting-2025-05-21-16-14-41-449                                          

2025-05-21 16:14:43 Starting - Starting the training job...
2025-05-21 16:15:07 Starting - Preparing the instances for training...
2025-05-21 16:15:42 Downloading - Downloading the training image.........
2025-05-21 16:17:14 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
Running custom environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
[05/21/2025 16:17:32 INFO 140328105305920] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'_kvstore': 'auto', '_num_gpus': 'auto', '_num_kv_servers': 'auto', '_tuning_objective_metric': '', 'cardinality': 'auto', 'dropout_rate': '0.10', 'early_stopping_patience': '', 'embedding_dimension': '10', 'learning_rate': '0.001', 'likeli

In [338]:
estimator = sagemaker.estimator.Estimator.attach('lilipink-forecasting-2025-05-21-16-14-41-449',sagemaker_session=sagemaker_session)


2025-05-21 16:23:49 Starting - Preparing the instances for training
2025-05-21 16:23:49 Downloading - Downloading the training image
2025-05-21 16:23:49 Training - Training image download completed. Training in progress.
2025-05-21 16:23:49 Uploading - Uploading generated training model
2025-05-21 16:23:49 Completed - Training job completed


In [339]:
predictor = estimator.deploy(
    initial_instance_count=1, instance_type="ml.m5.large", predictor_cls=DeepARPredictor
)

[05/21/25 11:27:16] INFO     Creating model with name: lilipink-forecasting-2025-05-21-16-27-15-927 ]8;id=890145;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=635645;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=975649;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=110072;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#6019\6019]8;;\
                             lilipink-forecasting-2025-05-21-16-27-15-927                                          

[05/21/25 11:27:17] INFO     Creating endpoint with name                                            ]8;id=559989;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=975198;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4841\4841]8;;\
                             lilipink-forecasting-2025-05-21-16-27-15-927                                          

-----------!

In [340]:
timeseries[6].head(1)

,material,Tipo_Producto,segmento_producto,supergrupo_producto,grupo_producto,subgrupo_producto,cantidad,month,quarter
2021-08-01,20001016001,0,0,0,0,0,25.0,8,3


In [341]:
horizon_pred=6
i=6
predictor.predict(
    ts = timeseries_list[i][:-horizon_pred], 
    cat=vectores_cat[i],
    dynamic_feat=dynamic_list[i],
    quantiles=[0.1, 0.5, 0.9],
)

,0.1,0.5,0.9
2024-11-01,-0.145210,5.119237,9.403378
2024-12-01,-7.877854,-1.599877,3.847554
2025-01-01,-1.344202,3.692548,9.995129
2025-02-01,-4.450853,0.291701,4.337483
2025-03-01,-0.706475,6.791806,12.815763
2025-04-01,-3.520440,3.702541,13.981384


In [342]:
timeseries_list[6].tail(6)

2024-11-01    14.0
2024-12-01    73.0
2025-01-01    26.0
2025-02-01    12.0
2025-03-01    23.0
2025-04-01    29.0
Freq: MS, Name: cantidad, dtype: float64

In [ ]:
### 3ER ENTRENAMIENTO ###

In [35]:
def convertir_cantidad_a_entero(lista_dataframes):
    """
    Convierte los valores de la columna 'cantidad' de cada dataframe a enteros,
    redondeando al entero más cercano.
    
    Args:
        lista_dataframes: Lista de dataframes con una columna 'cantidad'
    
    Returns:
        Una nueva lista de dataframes con los valores de 'cantidad' convertidos a enteros
    """
    import pandas as pd
    import numpy as np
    
    # Lista que almacenará los dataframes modificados
    lista_modificada = []
    
    for i, df in enumerate(lista_dataframes):
        # Crear una copia para no modificar el original
        df_modificado = df.copy()
        
        # Verificar que exista la columna 'cantidad'
        if 'cantidad' not in df_modificado.columns:
            print(f"Advertencia: Dataframe {i} no tiene la columna 'cantidad'. Se añadirá sin cambios.")
            lista_modificada.append(df_modificado)
            continue
        
        # Convertir a entero redondeando al entero más cercano
        try:
            # Primero verificar si hay valores nulos y manejarlos
            if df_modificado['cantidad'].isna().any():
                # Mantener los valores nulos como son
                mask = df_modificado['cantidad'].notna()
                df_modificado.loc[mask, 'cantidad'] = np.round(df_modificado.loc[mask, 'cantidad']).astype(int)
            else:
                # Si no hay valores nulos, convertir directamente
                df_modificado['cantidad'] = np.round(df_modificado['cantidad']).astype(int)
                
            lista_modificada.append(df_modificado)
            
        except Exception as e:
            print(f"Error al convertir valores en Dataframe {i}: {e}. Se añadirá sin cambios.")
            lista_modificada.append(df_modificado)
    
    return lista_modificada

In [36]:
timeseries = convertir_cantidad_a_entero(timeseries)

In [43]:
combined_df = pd.concat(timeseries, keys=range(len(timeseries)))

# Guardar con el índice (que ahora será MultiIndex)
combined_df.to_csv('timeseries_combined.csv', index=True)

In [345]:
vectores_target = extraer_vectores_cantidad(timeseries)

In [346]:
def crear_diccionarios_test(start, vectores_target, vectores_cat, vectores_dynamic):
    """
    Crea una lista de diccionarios con la estructura requerida para entrenamiento,
    donde start son las fechas de inicio de cada serie (un valor por dataframe).
    
    Args:
        start: Lista de strings con las fechas de inicio (un valor por dataframe)
        vectores_target: Lista de vectores con los valores de 'cantidad' para cada serie (como enteros)
        vectores_cat: Lista de vectores con las características categóricas
        vectores_dynamic: Lista de tuplas (month_vector, quarter_vector) con características dinámicas
    
    Returns:
        Lista de diccionarios con la estructura {start, target, cat, dynamic_feat}
    """
    import numpy as np
    
    def convert_nans_to_string(values_list):
        """Convierte valores NaN en la lista a 'NaN' como string, mantiene enteros como enteros"""
        return ['NaN' if (isinstance(x, float) and np.isnan(x)) else int(x) for x in values_list]
    
    diccionarios = []
    
    # Verificar que tengamos el mismo número de series en todas las listas
    num_series = len(start)
    if not (num_series == len(vectores_target) == len(vectores_cat) == len(vectores_dynamic)):
        print(f"Error: Las listas tienen diferentes longitudes - start: {len(start)}, target: {len(vectores_target)}, " 
              f"cat: {len(vectores_cat)}, dynamic: {len(vectores_dynamic)}")
        return []
    
    for i in range(num_series):
        # Verificar que haya datos para esta serie
        if not vectores_target[i]:
            print(f"Advertencia: Serie {i} no tiene valores target. Se omitirá.")
            continue
        
        # Obtener los datos para esta serie
        fecha_inicio = start[i]
        target_data = vectores_target[i]
        cat_data = vectores_cat[i]
        month_vector, quarter_vector = vectores_dynamic[i]
        
        # Verificar longitudes de vectors target y dynamic
        if len(target_data) != len(month_vector) or len(target_data) != len(quarter_vector):
            print(f"Advertencia: Serie {i} tiene longitudes inconsistentes - target: {len(target_data)}, "
                  f"month: {len(month_vector)}, quarter: {len(quarter_vector)}")
        
        # Crear el diccionario
        diccionario = {
            "start": fecha_inicio,
            "target": convert_nans_to_string(target_data),
            "cat": cat_data,
            "dynamic_feat": [month_vector, quarter_vector]  # Usar valores originales sin normalizar
        }
        
        diccionarios.append(diccionario)
    
    return diccionarios

In [347]:
test = crear_diccionarios_test(start,vectores_target,vectores_cat,vectores_dynamic)

In [348]:
def crear_diccionarios_entrenamiento(start, vectores_target, vectores_cat, vectores_dynamic, puntos_a_excluir=6):
    """
    Crea una lista de diccionarios excluyendo los últimos 'puntos_a_excluir' valores de 
    target y dynamic_feat para cada serie.
    
    Args:
        start: Lista de strings con las fechas de inicio (un valor por dataframe)
        vectores_target: Lista de vectores con los valores de 'cantidad' para cada serie
        vectores_cat: Lista de vectores con las características categóricas
        vectores_dynamic: Lista de tuplas (month_vector, quarter_vector) con características dinámicas
        puntos_a_excluir: Número de puntos a excluir del final de las series (default=6)
    
    Returns:
        Lista de diccionarios con la estructura {start, target, cat, dynamic_feat}
    """
    import numpy as np
    
    def convert_nans_to_string(values_list):
        """Convierte valores NaN en la lista a 'NaN' como string, mantiene enteros como enteros"""
        return ['NaN' if (isinstance(x, float) and np.isnan(x)) else int(x) for x in values_list]
    
    diccionarios = []
    
    # Verificar que tengamos el mismo número de series en todas las listas
    num_series = len(start)
    if not (num_series == len(vectores_target) == len(vectores_cat) == len(vectores_dynamic)):
        print(f"Error: Las listas tienen diferentes longitudes - start: {len(start)}, target: {len(vectores_target)}, " 
              f"cat: {len(vectores_cat)}, dynamic: {len(vectores_dynamic)}")
        return []
    
    for i in range(num_series):
        # Verificar que haya datos para esta serie
        if not vectores_target[i]:
            print(f"Advertencia: Serie {i} no tiene valores target. Se omitirá.")
            continue
        
        # Obtener los datos para esta serie
        fecha_inicio = start[i]
        target_data = vectores_target[i]
        cat_data = vectores_cat[i]
        month_vector, quarter_vector = vectores_dynamic[i]
        
        # Verificar que hay suficientes puntos para excluir
        if len(target_data) <= puntos_a_excluir:
            print(f"Advertencia: Serie {i} tiene menos puntos ({len(target_data)}) que los requeridos a excluir ({puntos_a_excluir}). Se omitirá.")
            continue
        
        # Excluir los últimos 'puntos_a_excluir' valores
        target_data_recortado = target_data[:-puntos_a_excluir]
        month_vector_recortado = month_vector[:-puntos_a_excluir]
        quarter_vector_recortado = quarter_vector[:-puntos_a_excluir]
        
        # Verificar longitudes de vectors target y dynamic después del recorte
        if len(target_data_recortado) != len(month_vector_recortado) or len(target_data_recortado) != len(quarter_vector_recortado):
            print(f"Advertencia: Serie {i} tiene longitudes inconsistentes después del recorte - target: {len(target_data_recortado)}, "
                  f"month: {len(month_vector_recortado)}, quarter: {len(quarter_vector_recortado)}")
            # Ajustar a la longitud mínima
            min_len = min(len(target_data_recortado), len(month_vector_recortado), len(quarter_vector_recortado))
            target_data_recortado = target_data_recortado[:min_len]
            month_vector_recortado = month_vector_recortado[:min_len]
            quarter_vector_recortado = quarter_vector_recortado[:min_len]
        
        # Crear el diccionario
        diccionario = {
            "start": fecha_inicio,
            "target": convert_nans_to_string(target_data_recortado),
            "cat": cat_data,
            "dynamic_feat": [month_vector_recortado, quarter_vector_recortado]
        }
        
        diccionarios.append(diccionario)
    
    return diccionarios

In [349]:
train = crear_diccionarios_entrenamiento(start,vectores_target, vectores_cat, vectores_dynamic)

In [ ]:
%%time
write_dicts_to_file("data_json/mensual_modificado_int/train.json", train)
write_dicts_to_file("data_json/mensual_modificado_int/test.json", test)

CPU times: total: 0 ns
Wall time: 0 ns


In [385]:
bucket_name = "forecasting-mensual-15-v4"
create_bucket(bucket_name)

Bucket S3 'forecasting-mensual-15-v4' creado exitosamente en la región por defecto


True

In [386]:
s3_bucket = bucket_name  # replace with an existing bucket if needed
s3_bucket_prefix = (
        "lilipink"  
    )
default_bucket_prefix = sagemaker_session.default_bucket_prefix
if default_bucket_prefix:
    s3_prefix = f"{default_bucket_prefix}/{s3_bucket_prefix}"
else:
    s3_prefix = s3_bucket_prefix

role = "arn:aws:iam::844598627082:role/service-role/AmazonSageMaker-ExecutionRole-20250513T105052"  # IAM role to use by SageMaker
region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output".format(s3_bucket, s3_prefix)

In [ ]:
local_file = 'data_json/mensual_modificado_int/'
copy_to_s3(local_file + 'train.json', s3_data_path + "/train/train.json",override=True)
copy_to_s3(local_file + 'test.json', s3_data_path + "/test/test.json",override=True)

Uploading file to s3://forecasting-mensual-15-v4/lilipink/data/train/train.json
Uploading file to s3://forecasting-mensual-15-v4/lilipink/data/test/test.json


In [395]:
freq= "M"
context_length = 18
prediction_length = 6
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "learning_rate": "1E-1",
    "likelihood":"negative-binomial",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
}

In [396]:
estimator.set_hyperparameters(**hyperparameters)

In [398]:
%%time
data_channels = {"train": "{}/train/".format(s3_data_path), "test": "{}/test/".format(s3_data_path)}

estimator.fit(inputs=data_channels, wait=True)

[05/21/25 17:50:26] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=946621;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=358654;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=190197;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=393617;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#1042\1042]8;;\
                             lilipink-forecasting-2025-05-21-22-50-26-760                                          

2025-05-21 22:50:28 Starting - Starting the training job...
2025-05-21 22:50:44 Starting - Preparing the instances for training...
2025-05-21 22:51:22 Downloading - Downloading the training image.........
2025-05-21 22:53:08 Training - Training image download completed. Training in progress...Docker entrypoint called with argument(s): train
Running default environment configuration script
Running custom environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
[05/21/2025 22:53:25 INFO 140039678244672] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'_kvstore': 'auto', '_num_gpus': 'auto', '_num_kv_servers': 'auto', '_tuning_objective_metric': '', 'cardinality': 'auto', 'dropout_rate': '0.10', 'early_stopping_patience': '', 'embedding_dimension': '10', 'learning_rate': '0.001', 'likel

In [399]:
estimator = sagemaker.estimator.Estimator.attach('lilipink-forecasting-2025-05-21-22-50-26-760',sagemaker_session=sagemaker_session)


2025-05-21 22:55:38 Starting - Preparing the instances for training
2025-05-21 22:55:38 Downloading - Downloading the training image
2025-05-21 22:55:38 Training - Training image download completed. Training in progress.
2025-05-21 22:55:38 Uploading - Uploading generated training model
2025-05-21 22:55:38 Completed - Training job completed


In [400]:
predictor = estimator.deploy(
    initial_instance_count=1, instance_type="ml.m5.large", predictor_cls=DeepARPredictor
)

[05/21/25 17:57:50] INFO     Creating model with name: lilipink-forecasting-2025-05-21-22-57-50-381 ]8;id=222346;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=403041;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4094\4094]8;;\

[05/21/25 17:57:51] INFO     Creating endpoint-config with name                                     ]8;id=238830;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=813013;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#6019\6019]8;;\
                             lilipink-forecasting-2025-05-21-22-57-50-381                                          

                    INFO     Creating endpoint with name                                            ]8;id=503750;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=857256;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4841\4841]8;;\
                             lilipink-forecasting-2025-05-21-22-57-50-381                                          

----------!

In [401]:
timeseries_list = convertir_a_series(timeseries)

In [402]:
horizon_pred=6
i=2
predictor.predict(
    ts = timeseries_list[i][:-horizon_pred], 
    cat=vectores_cat[i],
    dynamic_feat=dynamic_list[i],
    quantiles=[0.1, 0.5, 0.9],
)

,0.1,0.5,0.9
2024-11-01,4.0,9.0,17.0
2024-12-01,12.0,26.0,40.0
2025-01-01,8.0,16.0,24.0
2025-02-01,8.0,14.0,19.0
2025-03-01,2.0,5.0,16.0
2025-04-01,3.0,8.0,18.0


In [403]:
timeseries_list[i].tail(6)

2024-11-01    30
2024-12-01    36
2025-01-01     6
2025-02-01    11
2025-03-01     4
2025-04-01    18
Freq: MS, Name: cantidad, dtype: int32

In [ ]:
##########################

In [409]:
freq= "M"
context_length = 18
prediction_length = 6
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "likelihood": "student-T",
    "learning_rate": "0.001",
}
estimator.set_hyperparameters(**hyperparameters)

In [410]:
hyperparameter_ranges = {
    #"likelihood": CategoricalParameter(["gaussian", "negative-binomial", "student-T"]),
    "mini_batch_size": IntegerParameter(32,512),
    "num_cells": IntegerParameter(30, 200),  # Rango amplio
    #"learning_rate": ContinuousParameter(0.0001, 0.1),
    #"num_layers": IntegerParameter(1, 4),
}

In [411]:
objective_metric_name = "test:RMSE"

In [412]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    max_jobs=10,
    strategy="Bayesian",
    objective_type="Minimize",
    max_parallel_jobs=2,
    early_stopping_type="Auto",
)

In [414]:
tuner.fit({"train": "{}/train/".format(s3_data_path) , "test": "{}/test/".format(s3_data_path)}, include_cls_metadata=False)
tuner.wait()

[05/21/25 18:18:47] INFO     Creating hyperparameter tuning job with name:                          ]8;id=517022;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=312894;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#3383\3383]8;;\
                             forecasting-deepar-250521-1818                                                        

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
estimator = sagemaker.estimator.Estimator.attach('forecasting-deepar-250521-1818-009-87cebbdc',sagemaker_session=sagemaker_session)

In [416]:
predictor = estimator.deploy(
    initial_instance_count=1, instance_type="ml.m5.large", predictor_cls=DeepARPredictor
)

[05/21/25 21:58:37] INFO     Creating model with name: forecasting-deepar-2025-05-22-02-58-36-412   ]8;id=476931;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=454284;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4094\4094]8;;\

[05/21/25 21:58:38] INFO     Creating endpoint-config with name                                     ]8;id=808770;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=504010;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#6019\6019]8;;\
                             forecasting-deepar-2025-05-22-02-58-36-412                                            

                    INFO     Creating endpoint with name forecasting-deepar-2025-05-22-02-58-36-412 ]8;id=479013;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=82651;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4841\4841]8;;\

------------!

In [442]:
timeseries[0].tail(6)

,material,Tipo_Producto,segmento_producto,supergrupo_producto,grupo_producto,subgrupo_producto,cantidad,month,quarter
2024-11-01,20000337001,0,0,0,0,0,33,11,4
2024-12-01,20000337001,0,0,0,0,0,44,12,4
2025-01-01,20000337001,0,0,0,0,0,17,1,1
2025-02-01,20000337001,0,0,0,0,0,10,2,1
2025-03-01,20000337001,0,0,0,0,0,14,3,1
2025-04-01,20000337001,0,0,0,0,0,4,4,2


In [435]:
materiales = [df['material'].unique()[0] for df in timeseries]

In [441]:
materiales[0]

20000337001

In [431]:
horizon_pred=6
i=0
predictor.predict(
    ts = timeseries_list[i][:-horizon_pred], 
    cat=vectores_cat[i],
    dynamic_feat=dynamic_list[i],
    quantiles=[0.1, 0.5, 0.9],
)

,0.1,0.5,0.9
2024-11-01,10.772312,13.156721,15.081972
2024-12-01,17.859997,20.791840,23.978069
2025-01-01,7.654194,13.237596,17.679573
2025-02-01,-2.515227,14.570486,43.232300
2025-03-01,-14.742406,2.468850,19.240965
2025-04-01,-15.199956,-0.681237,19.299246


In [433]:
timeseries_list[i].tail(6)

2024-11-01    33
2024-12-01    44
2025-01-01    17
2025-02-01    10
2025-03-01    14
2025-04-01     4
Freq: MS, Name: cantidad, dtype: int32

In [443]:
def generar_predicciones_por_material(materiales, timeseries_list, vectores_cat, dynamic_list, predictor, horizon_pred=6):
    """
    Genera predicciones para cada material y las devuelve en un diccionario.
    
    Args:
        materiales: Lista con los nombres de materiales
        timeseries_list: Lista de series temporales
        vectores_cat: Lista de vectores categóricos
        dynamic_list: Lista de features dinámicas
        predictor: Modelo predictor entrenado
        horizon_pred: Horizonte de predicción (default: 6)
    
    Returns:
        dict: Diccionario con materiales como keys y dataframes de predicciones como values
    """
    predicciones_dict = {}
    
    for i in range(len(materiales)):
        material = materiales[i]
        
        # Generar predicción para el índice i
        prediccion = predictor.predict(
            ts = timeseries_list[i][:-horizon_pred], 
            cat=vectores_cat[i],
            dynamic_feat=dynamic_list[i],
            quantiles=[0.1, 0.5, 0.9],
        )
        
        # Guardar en el diccionario
        predicciones_dict[material] = prediccion
        
    return predicciones_dict

In [444]:
predicciones_por_material = generar_predicciones_por_material(
    materiales=materiales,
    timeseries_list=timeseries_list,
    vectores_cat=vectores_cat,
    dynamic_list=dynamic_list,
    predictor=predictor,
    horizon_pred=6
)

In [445]:
predicciones_por_material.keys()

dict_keys([20000337001, 20000400003, 20000815002, 20000815003, 20000837001, 20000837002, 20001016001, 20001374001, 20003147001, 20003257001, 20003257002, 20003257004, 20008046001, 25109225001, 25109232001])

In [446]:
predicciones_por_material[20000337001]

,0.1,0.5,0.9
2024-11-01,11.174452,13.033033,15.327240
2024-12-01,17.000874,20.702934,24.452698
2025-01-01,7.063230,12.199194,17.474936
2025-02-01,-5.257400,16.376047,34.438889
2025-03-01,-14.730208,1.734847,24.478224
2025-04-01,-19.101561,1.368773,17.366959


In [462]:
def exportar_predicciones_consolidado(predicciones_dict, nombre_archivo="mensual_modificado_test.xlsx"):
    """
    Exporta todas las predicciones con Material y Fecha como primeras dos columnas.
    """
    
    dfs_list = []
    
    for material, prediccion in predicciones_dict.items():
        try:
            # Convertir a DataFrame
            if hasattr(prediccion, 'to_dataframe'):
                df_pred = prediccion.to_dataframe()
            elif hasattr(prediccion, 'to_pandas'):
                df_pred = prediccion.to_pandas()
            else:
                df_pred = prediccion
            
            # Resetear índice y renombrar a 'Fecha'
            df_pred = df_pred.reset_index()
            date_col = df_pred.columns[0]
            df_pred = df_pred.rename(columns={date_col: 'Fecha'})
            
            # Formatear fechas
            df_pred['Fecha'] = pd.to_datetime(df_pred['Fecha']).dt.strftime('%Y-%m-%d')
            
            # Agregar columna de material
            df_pred['Material'] = material
            
            dfs_list.append(df_pred)
            
        except Exception as e:
            print(f"Error procesando {material}: {e}")
    
    # Concatenar todos los DataFrames
    if dfs_list:
        df_final = pd.concat(dfs_list, ignore_index=True)
        
        # Reorganizar columnas: Material, Fecha, luego el resto en orden
        other_cols = [col for col in df_final.columns if col not in ['Material', 'Fecha']]
        cols = ['Material', 'Fecha'] + other_cols
        df_final = df_final[cols]
        
        # Exportar
        df_final.to_excel(nombre_archivo, index=False)
        print(f"Archivo guardado: {nombre_archivo}")
        print(f"Orden de columnas: {list(df_final.columns)}")
        print("Formato de fecha: YYYY-MM-DD")
    else:
        print("Error: No se pudieron procesar las predicciones")

# Ejecutar
exportar_predicciones_consolidado(predicciones_por_material, "mensual_modificado_test.xlsx")

Archivo guardado: mensual_modificado_test.xlsx
Orden de columnas: ['Material', 'Fecha', '0.1', '0.5', '0.9']
Formato de fecha: YYYY-MM-DD


In [ ]:
##################### ENTRENAMIENTO CON TODA LA DATA CON HYPERPARÁMETROS OPTIMIZADOS #####################
#DATA TEST (COMPLETA) --> PARA TRAINING

In [456]:
estimator = sagemaker.estimator.Estimator(
    image_uri=image_name,
    sagemaker_session=sagemaker_session,
    role=role,
    instance_count=1,
    instance_type="ml.c4.2xlarge",
   # use_spot_instances=True,
   # max_run=1800,  # max training time in seconds
   # max_wait=1800,  # seconds to wait for spot instance
    base_job_name="lilipink-forecasting",
    output_path=s3_output_path,
)

In [457]:
freq= "M"
context_length = 18
prediction_length = 6
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "learning_rate": "1E-3",
    "likelihood":"student-T",
    "mini_batch_size": "449",
    "num_cells": "56",  # Rango amplio
    #"learning_rate": "0.001",
    "num_layers": "2",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    
}

In [458]:
estimator.set_hyperparameters(**hyperparameters)

In [459]:
%%time
data_channels = {"train": "{}/test/".format(s3_data_path)}

estimator.fit(inputs=data_channels, wait=True)

[05/21/25 22:48:29] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=842486;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=413242;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[05/21/25 22:48:30] INFO     Creating training-job with name:                                       ]8;id=168109;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=613742;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#1042\1042]8;;\
                             lilipink-forecasting-2025-05-22-03-48-30-015                                          

2025-05-22 03:48:32 Starting - Starting the training job...
2025-05-22 03:49:08 Downloading - Downloading input data...
2025-05-22 03:49:23 Downloading - Downloading the training image.........
2025-05-22 03:51:19 Training - Training image download completed. Training in progress....Docker entrypoint called with argument(s): train
Running default environment configuration script
Running custom environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
[05/22/2025 03:51:39 INFO 140041726678848] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'_kvstore': 'auto', '_num_gpus': 'auto', '_num_kv_servers': 'auto', '_tuning_objective_metric': '', 'cardinality': 'auto', 'dropout_rate': '0.10', 'early_stopping_patience': '', 'embedding_dimension': '10', 'learning_rate': '0.001', 'likelihood': 's

In [49]:
estimator = sagemaker.estimator.Estimator.attach('lilipink-forecasting-2025-05-22-03-48-30-015',sagemaker_session=sagemaker_session)


2025-05-22 04:04:15 Starting - Preparing the instances for training
2025-05-22 04:04:15 Downloading - Downloading the training image
2025-05-22 04:04:15 Training - Training image download completed. Training in progress.
2025-05-22 04:04:15 Uploading - Uploading generated training model
2025-05-22 04:04:15 Completed - Training job completed


In [17]:
predictor = estimator.deploy(
    initial_instance_count=1, instance_type="ml.m5.large", predictor_cls=DeepARPredictor
)

[05/22/25 00:37:32] INFO     Creating model with name: lilipink-forecasting-2025-05-22-05-37-31-368 ]8;id=770152;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=687007;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=41190;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=613768;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#6019\6019]8;;\
                             lilipink-forecasting-2025-05-22-05-37-31-368                                          

[05/22/25 00:37:33] INFO     Creating endpoint with name                                            ]8;id=336164;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=933419;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4841\4841]8;;\
                             lilipink-forecasting-2025-05-22-05-37-31-368                                          

------------!

In [53]:
def timeseries_to_excel(timeseries_list, output_filename='timeseries_export.xlsx'):
    """
    Exporta los últimos 6 puntos de una lista de dataframes timeseries a un archivo Excel.
    
    Parámetros:
    timeseries_list (list): Lista de DataFrames que contienen columnas 'material' y 'cantidad'
    output_filename (str): Nombre del archivo Excel de salida
    
    Returns:
    str: Ruta del archivo creado
    """
    
    # Lista para almacenar todos los datos procesados
    all_data = []
    
    # Procesar cada dataframe en la lista
    for i, df in enumerate(timeseries_list):
        # Verificar que el dataframe tiene las columnas necesarias
        if 'material' not in df.columns or 'cantidad' not in df.columns:
            print(f"Advertencia: DataFrame {i+1} no tiene las columnas 'material' y/o 'cantidad'")
            continue
        
        # Tomar los últimos 6 puntos
        last_6_points = df.tail(6)
        
        # Agregar solo las columnas necesarias
        data_to_add = last_6_points[['material', 'cantidad']].copy()
        
        # Opcional: agregar una columna para identificar de qué dataframe viene
        data_to_add['dataframe_origen'] = f'TS_{i+1}'
        
        all_data.append(data_to_add)
    
    # Combinar todos los datos
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        
        # Si no quieres la columna de origen, descomenta la siguiente línea:
        # final_df = final_df[['material', 'cantidad']]
        
        # Exportar a Excel
        final_df.to_excel(output_filename, index=False)
        print(f"Archivo '{output_filename}' creado exitosamente con {len(final_df)} registros")
        
        return output_filename
    else:
        print("No se encontraron datos válidos para exportar")
        return None

In [54]:
archivo_creado = timeseries_to_excel(timeseries, 'data_real.xlsx')

Archivo 'data_real.xlsx' creado exitosamente con 90 registros


In [463]:
materiales = [df['material'].unique()[0] for df in timeseries]

In [464]:
horizon_pred=6
i=0
predictor.predict(
    ts = timeseries_list[i][:-horizon_pred], 
    cat=vectores_cat[i],
    dynamic_feat=dynamic_list[i],
    quantiles=[0.1, 0.5, 0.9],
)

,0.1,0.5,0.9
2024-11-01,28.570580,33.706036,37.759369
2024-12-01,40.106171,44.941391,48.891117
2025-01-01,15.922655,17.287342,18.594730
2025-02-01,9.118885,10.537629,12.171485
2025-03-01,5.690591,10.755075,14.273312
2025-04-01,2.132280,6.541682,10.474480


In [465]:
timeseries_list[i].tail(6)

2024-11-01    33
2024-12-01    44
2025-01-01    17
2025-02-01    10
2025-03-01    14
2025-04-01     4
Freq: MS, Name: cantidad, dtype: int32

In [466]:
predicciones_por_material = generar_predicciones_por_material(
    materiales=materiales,
    timeseries_list=timeseries_list,
    vectores_cat=vectores_cat,
    dynamic_list=dynamic_list,
    predictor=predictor,
    horizon_pred=6
)

In [467]:
predicciones_por_material.keys()

dict_keys([20000337001, 20000400003, 20000815002, 20000815003, 20000837001, 20000837002, 20001016001, 20001374001, 20003147001, 20003257001, 20003257002, 20003257004, 20008046001, 25109225001, 25109232001])

In [472]:
predicciones_por_material[20000400003]

,0.1,0.5,0.9
2024-11-01,3.511388,5.638802,7.351160
2024-12-01,10.944141,13.902410,17.068285
2025-01-01,4.117705,4.820570,5.672948
2025-02-01,2.737851,3.517854,4.493963
2025-03-01,2.222531,4.483961,6.030576
2025-04-01,-1.047628,1.319560,3.686026


In [469]:
exportar_predicciones_consolidado(predicciones_por_material, "mensual_modificado_completo.xlsx")

Archivo guardado: mensual_modificado_completo.xlsx
Orden de columnas: ['Material', 'Fecha', '0.1', '0.5', '0.9']
Formato de fecha: YYYY-MM-DD


In [474]:
predictor.delete_model()
predictor.delete_endpoint()

[05/21/25 23:28:26] INFO     Deleting model with name: lilipink-forecasting-2025-05-22-04-06-24-816 ]8;id=565771;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=19646;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#5356\5356]8;;\

[05/21/25 23:28:27] INFO     Deleting endpoint configuration with name:                             ]8;id=136362;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=750951;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4995\4995]8;;\
                             lilipink-forecasting-2025-05-22-04-06-24-816                                          

                    INFO     Deleting endpoint with name:                                           ]8;id=660670;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=695552;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4985\4985]8;;\
                             lilipink-forecasting-2025-05-22-04-06-24-816                                          